In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('..//')
from utils_mitgcm import open_mitgcm_ds_from_config

In [ ]:
def compute_ke_snapshot(uvel, vvel, wvel, dx, dy, dz, rho=1000.0):
    ke = 0.5 * rho * (uvel ** 2 + vvel ** 2 + wvel ** 2) * dx * dy * dz  # This gives J per cell

    return ke / 1e6  # Convert to MJ

In [ ]:
model = 'geneva_dummy_extended'
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
grid_resolution = 200
ds['YC'] = np.arange(1, len(ds['YC']) + 1) * grid_resolution - grid_resolution / 2
ds['XC'] = np.arange(1, len(ds['XC']) + 1) * grid_resolution - grid_resolution / 2
ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

In [ ]:
%matplotlib inline
t=-1
plt.close('all')
plt.figure(figsize=(15,7))
ds_sel = ds['THETA'].isel(Z=0, time=t)
ds_sel.where(ds_sel != 0, np.nan).plot()
plt.title(ds_sel.time.values)
#plt.savefig(r"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/figures/transect_location.png")
plt.show()

In [ ]:
aligned_u = ds.UVEL.rename({'XG':'XC'})
aligned_u['XC'] = ds['XC']

aligned_v = ds.VVEL.rename({'YG':'YC'})
aligned_v['YC'] = ds['YC']

aligned_w = ds.WVEL.rename({'Zl':'Z'})
aligned_w['Z'] = ds['Z']

In [ ]:
ke_surf = compute_ke_snapshot(aligned_u.isel(Z=0), aligned_v.isel(Z=0), aligned_w.isel(Z=0), 200, 200, 0.5)

In [ ]:
ke_surf.sum(dim=['XC','YC']).plot()